# Overview

Once we have finished experiementation and found a good model, we want to operationalize it. 



In [8]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [1]:
from teradataml.dataframe.dataframe import DataFrame
from teradataml import create_context
import getpass

engine = create_context(host="3.238.151.85", username="AOA_MASTER", password=getpass.getpass("password"))

password ··········


### Define Training Function

The training function takes the following shape

```python
def train(context: ModelContext, **kwargs):
    aoa_create_context()
    
    # your training code
    
    # save your model
    joblib.dump(model, f"{context.artifact_output_path}/model.joblib")
    
    record_training_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [2]:
# %%writefile ../model_modules/training.py

from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from nyoka import xgboost_to_pmml
from teradataml import DataFrame
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)

import joblib


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]

    # read training dataset from Teradata and convert to pandas
    train_df = DataFrame.from_query(context.dataset_info.sql)
    train_pdf = train_df.to_pandas(all_rows=True)

    # split data into X and y
    X_train = train_pdf[feature_names]
    y_train = train_pdf[target_name]

    print("Starting training...")

    # fit model to training data
    model = Pipeline([('scaler', MinMaxScaler()),
                      ('xgb', XGBClassifier(eta=context.hyperparams["eta"],
                                            max_depth=context.hyperparams["max_depth"]))])

    model.fit(X_train, y_train)

    print("Finished training")

    # export model artefacts
    joblib.dump(model, f"{context.artifact_output_path}/model.joblib")

    # we can also save as pmml so it can be used for In-Vantage scoring etc.
    xgboost_to_pmml(pipeline=model, col_names=feature_names, target_name=target_name,
                    pmml_f_name=f"{context.artifact_output_path}/model.pmml")

    print("Saved trained model")

    from xgboost import plot_importance
    model["xgb"].get_booster().feature_names = feature_names
    plot_importance(model["xgb"].get_booster(), max_num_features=10)
    save_plot("feature_importance.png", context=context)

    feature_importance = model["xgb"].get_booster().get_score(importance_type="weight")

    record_training_stats(train_df,
                          features=feature_names,
                          predictors=[target_name],
                          categorical=[target_name],
                          importance=feature_importance,
                          context=context)


In [25]:
from aoa import ModelContext, DatasetInfo

# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the training dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
"""

feature_metadata =  {
    "database": "AOA_MASTER",
    "table": "aoa_feature_metadata"
}
hyperparams = {"max_depth": 5, "eta": 0.2}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]
 
dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)


ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="/tmp",
                   model_version="v1",
                   model_table="aoa_model_v1")

train(context=ctx)

INFO:aoa.util.connections:teradataml context already exists. Skipping create_context.
Starting training...
Finished training
Saved trained model
INFO:aoa.stats.stats:Computing training dataset statistics


<Figure size 432x288 with 0 Axes>

### Define Evaluation Function

The evaluation function takes the following shape

```python
def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    model = joblib.load(f"{context.artifact_input_path}/model.joblib")
    
    # your evaluation logic
    
    record_evaluation_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [4]:
# %%writefile ../model_modules/evaluation.py

from sklearn import metrics
from teradataml import DataFrame, copy_to_sql
from aoa import (
    record_evaluation_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)

import joblib
import json
import numpy as np
import pandas as pd


def evaluate(context: ModelContext, **kwargs):

    aoa_create_context()

    model = joblib.load(f"{context.artifact_input_path}/model.joblib")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]

    test_df = DataFrame.from_query(context.dataset_info.sql)
    test_pdf = test_df.to_pandas(all_rows=True)

    X_test = test_pdf[feature_names]
    y_test = test_pdf[target_name]

    print("Scoring")
    y_pred = model.predict(X_test)

    y_pred_tdf = pd.DataFrame(y_pred, columns=[target_name])
    y_pred_tdf["PatientId"] = test_pdf["PatientId"].values

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics.accuracy_score(y_test, y_pred)),
        'Recall': '{:.2f}'.format(metrics.recall_score(y_test, y_pred)),
        'Precision': '{:.2f}'.format(metrics.precision_score(y_test, y_pred)),
        'f1-score': '{:.2f}'.format(metrics.f1_score(y_test, y_pred))
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)

    metrics.plot_confusion_matrix(model, X_test, y_test)
    save_plot('Confusion Matrix', context=context)

    metrics.plot_roc_curve(model, X_test, y_test)
    save_plot('ROC Curve', context=context)

    # xgboost has its own feature importance plot support but lets use shap as explainability example
    import shap

    shap_explainer = shap.TreeExplainer(model['xgb'])
    shap_values = shap_explainer.shap_values(X_test)

    shap.summary_plot(shap_values, X_test, feature_names=feature_names,
                      show=False, plot_size=(12, 8), plot_type='bar')
    save_plot('SHAP Feature Importance', context=context)

    feature_importance = pd.DataFrame(list(zip(feature_names, np.abs(shap_values).mean(0))),
                                      columns=['col_name', 'feature_importance_vals'])
    feature_importance = feature_importance.set_index("col_name").T.to_dict(orient='records')[0]

    predictions_table = "evaluation_preds_tmp"
    copy_to_sql(df=y_pred_tdf, table_name=predictions_table, index=False, if_exists="replace", temporary=True)

    record_evaluation_stats(features_df=test_df,
                            predicted_df=DataFrame(predictions_table),
                            importance=feature_importance,
                            context=context)


In [24]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="/tmp",
                   artifact_input_path="/tmp",
                   model_version="v1",
                   model_table="aoa_model_v1")

evaluate(context=ctx)

INFO:aoa.util.connections:teradataml context already exists. Skipping create_context.
Scoring
DEBUG:matplotlib.colorbar:locator: <matplotlib.ticker.AutoLocator object at 0x7fc79f609940>
INFO:aoa.stats.stats:Computing evaluation dataset statistics


<Figure size 432x288 with 0 Axes>

<Figure size 864x576 with 0 Axes>

### Define Scoring Function

The scoring function takes the following shape

```python
def score(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    model = joblib.load(f"{context.artifact_input_path}/model.joblib")
    
    # your evaluation logic
    
    record_scoring_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [18]:
# %%writefile ../model_modules/scoring.py

from teradataml import copy_to_sql, DataFrame
from aoa import (
    record_scoring_stats,
    aoa_create_context,
    ModelContext
)

import joblib
import pandas as pd


def score(context: ModelContext, **kwargs):

    aoa_create_context()

    model = joblib.load(f"{context.artifact_input_path}/model.joblib")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    features_tdf = DataFrame.from_query(context.dataset_info.sql)
    features_pdf = features_tdf.to_pandas(all_rows=True)

    print("Scoring")
    predictions_pdf = model.predict(features_pdf[feature_names])

    print("Finished Scoring")

    # store the predictions
    predictions_pdf = pd.DataFrame(predictions_pdf, columns=[target_name])
    predictions_pdf[entity_key] = features_pdf.index.values
    # add job_id column so we know which execution this is from if appended to predictions table
    predictions_pdf["job_id"] = context.job_id

    # teradataml doesn't match column names on append.. and so to match / use same table schema as for byom predict
    # example (see README.md), we must add empty json_report column and change column order manually (v17.0.0.4)
    # CREATE MULTISET TABLE pima_patient_predictions
    # (
    #     job_id VARCHAR(255), -- comes from airflow on job execution
    #     PatientId BIGINT,    -- entity key as it is in the source data
    #     HasDiabetes BIGINT,   -- if model automatically extracts target
    #     json_report CLOB(1048544000) CHARACTER SET UNICODE  -- output of
    # )
    # PRIMARY INDEX ( job_id );
    predictions_pdf["json_report"] = ""
    predictions_pdf = predictions_pdf[["job_id", entity_key, target_name, "json_report"]]

    copy_to_sql(df=predictions_pdf,
                schema_name=context.dataset_info.predictions_database,
                table_name=context.dataset_info.predictions_table,
                index=False,
                if_exists="append")
    
    print("Saved predictions in Teradata")

    # calculate stats
    predictions_df = DataFrame.from_query(f"""
        SELECT 
            * 
        FROM {context.dataset_info.get_predictions_metadata_fqtn()} 
            WHERE job_id = '{context.job_id}'
    """)

    record_scoring_stats(features_df=features_tdf, predicted_df=predictions_df, context=context)


In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the scoring dataset 

sql = """
SELECT 
    F.*
FROM PIMA_PATIENT_FEATURES F 
    WHERE F.patientid MOD 5 = 0
"""

# where to store predictions
predictions = {
    "database": "AOA_MASTER",
    "table": "pima_patient_predictions_tmp"
}

import uuid
job_id=str(uuid.uuid4())

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata,
                           predictions=predictions)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="/tmp",
                   artifact_input_path="/tmp",
                   model_version="v1",
                   model_table="aoa_model_v1",
                   job_id=job_id)

score(context=ctx)

INFO:aoa.util.connections:teradataml context already exists. Skipping create_context.
Scoring
Finished Scoring
Saved predictions in Teradata
INFO:aoa.stats.stats:Computing scoring dataset statistics


### Define Model Metadata

Finally, create the configuration files.

Requirements file with the dependencies and versions

```
%%writefile ../model_modules/requirements.txt
xgboost==0.90
scikit-learn==0.24.2
shap==0.36.0
matplotlib==3.3.1
teradataml>=17.0.0.4
nyoka>=4.3.0
aoa==6.0.0
```

The hyper parameter configuration (defaults)
```
%%writefile ../config.json
{
   "hyperParameters": {
      "eta": 0.2,
      "max_depth": 6
   }
}
```